In [8]:
import jieba
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from hanziconv import HanziConv
import pickle
from gensim.models import word2vec
import pandas as pd

In [4]:
### word2vec model traning

fileTrainRead = []
with open("corpus.txt",encoding="utf-8") as fileTrainRaw:
    for line in fileTrainRaw:
        fileTrainRead.append(HanziConv.toTraditional(line)) # 簡轉繁

# 斷詞
fileTrainSeg=[]
for i in range(len(fileTrainRead)):
    fileTrainSeg.append([' '.join(list(jieba.cut(fileTrainRead[i][9:-11],cut_all=False)))])
# 因為會跑很久，檢核是否資料有持續在跑
    if i % 50000 == 0 :
        print(i)

# 將jieba的斷詞產出存檔
fileSegWordDonePath ='corpusSegDone.txt'
with open(fileSegWordDonePath,'wb') as fW:
    for i in range(len(fileTrainSeg)):
        fW.write(fileTrainSeg[i][0].encode('utf-8'))
        fW.write(b'\n')

from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

# Settings
seed = 666
sg = 0
window_size = 5
vector_size = 200
min_count = 1
workers = 4
epochs = 5
batch_words = 10000

train_data = word2vec.LineSentence('corpusSegDone.txt')
model = word2vec.Word2Vec(
    train_data,
    min_count=min_count,
    size=vector_size,
    workers=workers,
    iter=epochs,
    window=window_size,
    sg=sg,
    seed=seed,
    batch_words=batch_words
)

model.save('word2vec.model')

In [12]:
###logisregression model traning
#load word2vec model
word2vec_model = word2vec.Word2Vec.load('word2vec.model')


In [13]:
df_label = pd.read_excel("concreteness_labels.xlsx")

In [14]:
df_label

,Word,Word_ch,Bigram,Conc.Medium,Conc.Stantard deviation,Unknown,Total,Percent_known,SUBTLEX,Dom_Pos
0,angelfish,神仙魚,0,5.00,0.00,0,27,1.00,5,Noun
1,antelope,羚羊,0,5.00,0.00,0,28,1.00,50,Noun
2,apple,蘋果,0,5.00,0.00,0,30,1.00,1207,Noun
3,armchair,扶手椅,0,5.00,0.00,0,29,1.00,21,Noun
4,axe,斧頭,0,5.00,0.00,0,27,1.00,249,Noun
...,...,...,...,...,...,...,...,...,...,...
14587,judgment,判斷,0,1.68,1.33,0,28,1.00,839,Noun
14588,heaven,天堂,0,1.81,1.47,0,27,1.00,2887,Noun
14589,reincarnation,投胎,0,1.63,1.30,0,30,1.00,66,Noun
14590,altruism,利他主義,0,1.50,1.19,3,25,0.88,11,Noun


In [15]:
#decide if row is concrete or not
def concrete_judge(row):
    if row['Conc.Medium'] >= 4.7 and row["Conc.Stantard deviation"]<=1:
        return 1
    if row['Conc.Medium'] <= 3.0:
        return 0
    return None

In [16]:
df_label["is_concrete"]=df_label.apply(lambda row: concrete_judge(row), axis=1)
df_label=df_label.dropna()

In [17]:
df_label

,Word,Word_ch,Bigram,Conc.Medium,Conc.Stantard deviation,Unknown,Total,Percent_known,SUBTLEX,Dom_Pos,is_concrete
0,angelfish,神仙魚,0,5.00,0.00,0,27,1.00,5,Noun,1.0
1,antelope,羚羊,0,5.00,0.00,0,28,1.00,50,Noun,1.0
2,apple,蘋果,0,5.00,0.00,0,30,1.00,1207,Noun,1.0
3,armchair,扶手椅,0,5.00,0.00,0,29,1.00,21,Noun,1.0
4,axe,斧頭,0,5.00,0.00,0,27,1.00,249,Noun,1.0
...,...,...,...,...,...,...,...,...,...,...,...
14587,judgment,判斷,0,1.68,1.33,0,28,1.00,839,Noun,0.0
14588,heaven,天堂,0,1.81,1.47,0,27,1.00,2887,Noun,0.0
14589,reincarnation,投胎,0,1.63,1.30,0,30,1.00,66,Noun,0.0
14590,altruism,利他主義,0,1.50,1.19,3,25,0.88,11,Noun,0.0


In [19]:
# convert word to word vector and save to x_df
x_df=pd.DataFrame()
y_df=pd.DataFrame()
for index,row in df_label.iterrows():
    if index%300 ==0:
        print(index)
    try:
        word_vector=pd.DataFrame([word2vec_model[row["Word_ch"]]])
        x_df=x_df.append(word_vector)
        y_df=y_df.append(pd.DataFrame({row["is_concrete"]}))
    except KeyError:
        print("word {} not in word2vec_model".format(row["Word_ch"]))
        pass

0
word 神仙魚 not in word2vec_model
word 麵包圈 not in word2vec_model
word 自行車 not in word2vec_model
word 竊聽器 not in word2vec_model
word 捲餅 not in word2vec_model


c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


word 攝錄一體機 not in word2vec_model
word 胡蘿蔔 not in word2vec_model
word 紙板箱 not in word2vec_model
word 時鐘 not in word2vec_model
word 眼鏡蛇 not in word2vec_model
word 曲奇餅 not in word2vec_model
word 杯形餅 not in word2vec_model
word 除臭劑 not in word2vec_model
word 自動樓梯 not in word2vec_model
word 長笛 not in word2vec_model
word 蒼蠅拍 not in word2vec_model
word 灰獵犬 not in word2vec_model
word 漢堡包 not in word2vec_model
word 馬蹄鐵 not in word2vec_model
word 牛仔褲 not in word2vec_model
word 番茄醬 not in word2vec_model
word 棒糖 not in word2vec_model
word 瑪格麗塔 not in word2vec_model
word 馬騾 not in word2vec_model
word 連褲襪 not in word2vec_model
word 意大利辣味香腸 not in word2vec_model
word 板 not in word2vec_model
word 長袍 not in word2vec_model
word 薩克管 not in word2vec_model
word 防雪裝 not in word2vec_model
word 意大利面 not in word2vec_model
word 出租車 not in word2vec_model
word 拖拉機 not in word2vec_model
word 錄像帶 not in word2vec_model
word 錶帶 not in word2vec_model
word 知更鳥 not in word2vec_model
word 黑板 not in word2vec_model
word 起司蛋

word 天線 not in word2vec_model
word 卡祖笛 not in word2vec_model
word 鄰里 not in word2vec_model
word 夏南瓜 not in word2vec_model
word 腳手架 not in word2vec_model
word 營火 not in word2vec_model
word 理髮師 not in word2vec_model
word 髮型師 not in word2vec_model
word 鉤 not in word2vec_model
word 麵包 not in word2vec_model
word 天空之光 not in word2vec_model
word 翻筋斗 not in word2vec_model
word 棒子麵 not in word2vec_model
word 線 not in word2vec_model
word 拉鍊 not in word2vec_model
word 鵝卵石 not in word2vec_model
word 烘乾機 not in word2vec_model
word 百科全書 not in word2vec_model
word 蓋屋頂 not in word2vec_model
word 玄鳳鸚鵡 not in word2vec_model
word 麥克風 not in word2vec_model
word 獨木舟 not in word2vec_model
word 夾克 not in word2vec_model
word 滑板 not in word2vec_model
word 聯排別墅 not in word2vec_model
word 刨花板 not in word2vec_model
word 洗碗機 not in word2vec_model
word 綁腿 not in word2vec_model
word 石膏板 not in word2vec_model
word 單板滑雪 not in word2vec_model
word 美洲獅 not in word2vec_model
word 盒式磁帶 not in word2vec_model
word 莫卡辛 not i

word 天才 not in word2vec_model
word televangelism not in word2vec_model
word 註冊 not in word2vec_model
word 寡婦身份 not in word2vec_model
word 靈活性 not in word2vec_model
word 系統 not in word2vec_model
word 鑑賞 not in word2vec_model
word TECHNO not in word2vec_model
word 弦聲 not in word2vec_model
word 組合 not in word2vec_model
word 出版 not in word2vec_model
word ironbound not in word2vec_model
word 蒼白的臉 not in word2vec_model
9600
word 裂了 not in word2vec_model
word 貪食症 not in word2vec_model
word 關係 not in word2vec_model
word 關鍵詞 not in word2vec_model
word 上升氣流 not in word2vec_model
word 立體主義 not in word2vec_model
word 細木工 not in word2vec_model
word 催人淚下 not in word2vec_model
word 生物技術 not in word2vec_model
word desecrator not in word2vec_model
word 第一線 not in word2vec_model
word 製片人 not in word2vec_model
word 催化劑 not in word2vec_model
word highflier not in word2vec_model
word 歷史 not in word2vec_model
word 特立獨行 not in word2vec_model
word 神秘學者 not in word2vec_model
word 短程遊覽 not in word2vec_model
wor

word 毆打 not in word2vec_model
word 重新錄製 not in word2vec_model
word 康復 not in word2vec_model
word 複製 not in word2vec_model
word 演出 not in word2vec_model
word 和聲 not in word2vec_model
word 恢復 not in word2vec_model
word 彈震 not in word2vec_model
word flincher not in word2vec_model
word 注出 not in word2vec_model
word 征服 not in word2vec_model
word 聚光燈 not in word2vec_model
word 悲觀主義者 not in word2vec_model
word 回購 not in word2vec_model
word 對抗手段 not in word2vec_model
word 父權制 not in word2vec_model
word 佔用 not in word2vec_model
word 不適宜 not in word2vec_model
word 聽不到 not in word2vec_model
word 產卵 not in word2vec_model
word 專家 not in word2vec_model
word 無用功 not in word2vec_model
word 法律學家 not in word2vec_model
word 指示符 not in word2vec_model
word 營養不良 not in word2vec_model
word 百萬分之一 not in word2vec_model
word 大使的職位 not in word2vec_model
word 對比 not in word2vec_model
word 調製 not in word2vec_model
word 慈善機構 not in word2vec_model
word 相干性 not in word2vec_model
word 生物電 not in word2vec_model
word 緊急

word 持不同政見 not in word2vec_model
word 紊亂ETC not in word2vec_model
word 闖入者 not in word2vec_model
word 和平主義者 not in word2vec_model
word 誇張者 not in word2vec_model
word 溫和 not in word2vec_model
word 內分泌 not in word2vec_model
word 外邦人 not in word2vec_model
word 自身免疫性 not in word2vec_model
word 過度興奮 not in word2vec_model
word 試金石 not in word2vec_model
word 末日預言者 not in word2vec_model
word 識別碼 not in word2vec_model
word 賽季中期 not in word2vec_model
word 方向 not in word2vec_model
word 不情願 not in word2vec_model
word 託辭 not in word2vec_model
word 綜合徵 not in word2vec_model
word 兄弟會 not in word2vec_model
word 鐵桿 not in word2vec_model
word 遺產 not in word2vec_model
word dimwittedness not in word2vec_model
word 雙性戀 not in word2vec_model
word 管理人之職務 not in word2vec_model
word 長處 not in word2vec_model
word 表裡不一 not in word2vec_model
word 口頭禪 not in word2vec_model
word 兩星期 not in word2vec_model
word 性慾 not in word2vec_model
word 複審 not in word2vec_model
word 過敏症 not in word2vec_model
word 後果 not in word2v

word devilry not in word2vec_model
word 心理劇 not in word2vec_model
word 推斥 not in word2vec_model
word 重新定義 not in word2vec_model
word 鬆懈 not in word2vec_model
word 託管 not in word2vec_model
word orneriness not in word2vec_model
word 跑來跑去 not in word2vec_model
word 區別 not in word2vec_model
word slothfulness not in word2vec_model
word 再投資 not in word2vec_model
word 喜怒無常 not in word2vec_model
word niftiness not in word2vec_model
word 困惑 not in word2vec_model
word voraciousness not in word2vec_model
word 複數 not in word2vec_model
word demoralizer not in word2vec_model
word thingamajig not in word2vec_model
word 所有權 not in word2vec_model
word 困境 not in word2vec_model
word 富麗堂皇 not in word2vec_model
word 運動能力 not in word2vec_model
word 征服 not in word2vec_model
word 出現 not in word2vec_model
word 糊塗人 not in word2vec_model
word 復活 not in word2vec_model
word 限制 not in word2vec_model
word 和藹 not in word2vec_model
word FIB not in word2vec_model
word 蔑視 not in word2vec_model
12600
word 斯多葛派 not in wor

word 亞文化 not in word2vec_model
word 無政府狀態 not in word2vec_model
word 讀寫能力 not in word2vec_model
word 非公開 not in word2vec_model
word 不理解 not in word2vec_model
word gayness not in word2vec_model
word 不幸的事 not in word2vec_model
word 乏善可陳 not in word2vec_model
word 不確定 not in word2vec_model
word 滅絕人性 not in word2vec_model
word 自以為是 not in word2vec_model
word 肆無忌單 not in word2vec_model
word 不受歡迎 not in word2vec_model
word 專門知識 not in word2vec_model
word 準備就緒 not in word2vec_model
word 非事件 not in word2vec_model
word 排外主義 not in word2vec_model
word 虛張聲勢 not in word2vec_model
word 二元性 not in word2vec_model
word 地震學 not in word2vec_model
word 閱歷 not in word2vec_model
word 樂天派 not in word2vec_model
word 心理學 not in word2vec_model
word 頭韻 not in word2vec_model
word 污點 not in word2vec_model
word 完美主義 not in word2vec_model
word sneakiness not in word2vec_model
word 控制 not in word2vec_model
word 重新發現 not in word2vec_model
word 恆常 not in word2vec_model
word 大肆渲染 not in word2vec_model
word 雞毛蒜皮的小事 not 

word 鬥智斗勇 not in word2vec_model
word 喪失工作能力 not in word2vec_model
word 困擾 not in word2vec_model
word 神創論 not in word2vec_model
word 民族主義 not in word2vec_model
word 幫倒忙 not in word2vec_model
word 原教旨主義 not in word2vec_model
word 靈敏度 not in word2vec_model
word 利己主義 not in word2vec_model
word 合法 not in word2vec_model
word 一神教 not in word2vec_model
word 顯而易見 not in word2vec_model
word 任何方式 not in word2vec_model
word 表現主義 not in word2vec_model
word 虛偽 not in word2vec_model
word ID not in word2vec_model
word 複雜 not in word2vec_model
word illicitness not in word2vec_model
word 哄騙 not in word2vec_model
word 知性主義 not in word2vec_model
word 能力傾向 not in word2vec_model
word 鍾愛 not in word2vec_model
word changelessness not in word2vec_model
word 污穢 not in word2vec_model
word 抑制 not in word2vec_model
word 快活音樂 not in word2vec_model
word 胡言亂語 not in word2vec_model
word 不適合 not in word2vec_model
word 一致性 not in word2vec_model
word 回顧 not in word2vec_model
word 欣快症 not in word2vec_model
word 世說新語 not i

In [20]:
y_df[0].value_counts()

0.0    3465
1.0    1644
Name: 0, dtype: int64

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.25, random_state=0)

In [22]:
# all parameters not specified are set to their defaults
logisticRegr = LogisticRegression()

In [23]:
logisticRegr.fit(x_train, np.ravel(y_train))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
score = logisticRegr.score(x_test, y_test)
print(score)

0.8693270735524257


In [25]:
#save model
with open('logisticRefrModel.pickle', 'wb') as f:
    pickle.dump(logisticRegr, f)

In [26]:
#### traning end

In [27]:
### predict 

In [30]:
df_word=pd.read_excel("test_word.xlsx")

In [31]:
def predict(word):
    y_result=None
    try:
        word_vector=pd.DataFrame([word2vec_model[word]])
        y_result=logisticRegr.predict(word_vector)
    except KeyError:
        print("word '{}' not in dict".format(word))
        return None
    return int(y_result)

In [32]:
df_word["is_concrete"]=df_word["word"].apply(predict)

c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instea

word '畫面' not in dict
word '髮絲' not in dict
word '頭髮' not in dict
word '船隻' not in dict
word '鐘' not in dict


c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instea

word '家' not in dict
word '吧台' not in dict
word '吧臺' not in dict
word '波旁威士忌' not in dict
word '我喜歡' not in dict
word '那個女人' not in dict


c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instea

...

c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instea

word '新生活' not in dict
word '走向' not in dict
word '關係' not in dict
word '工作人員' not in dict
word '禮品盒' not in dict
word '緣由' not in dict
word '通知單' not in dict


c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instea

word '我的孩子' not in dict


c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [33]:
predict("測試")

c:\users\p20295\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


0